In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_json,col
from pyspark.sql.types import *
from os.path import abspath

spark = SparkSession\
        .builder\
        .appName("kafka-producer")\
        .master("local")\
        .config("hive.metastore.uris", "thrift://hive-metastore:9083")\
        .config("spark.sql.warehouse.dir", "hdfs://namenode:8020/user/hive/warehouse")\
        .config("spark.executor.memory", "1g")\
        .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.2.0")\
        .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1')\
        .config('spark.sql.shuffle.partitions', '1')\
        .enableHiveSupport()\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
sc = spark._sc

In [ ]:
lines = spark.read.csv("shakespeare/shakespeare.txt")
lines.show(5)

In [ ]:
# Write key-value data from a DataFrame to a specific Kafka topic specified in an option
ds = lines \
  .selectExpr("CAST(_c1 AS STRING) as key", "CAST(_c0 AS STRING) as value") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "broker:29092") \
  .option("topic", "wordcount") \
  .save()

